**tensorflow**



In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Input, layers, Model

print(tf.__version__)
print(tf.keras.__version__)

In [85]:
# model = tf.keras.Sequential()
# model.add(layers.Dense(32, activation='relu'))
# model.add(layers.Dense(32, activation='relu'))
# model.add(layers.Dense(10, activation='softmax'))

# model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
#              loss=tf.keras.losses.categorical_crossentropy,
#              metrics=[tf.keras.metrics.categorical_accuracy])

class CustomModel(tf.keras.Model):
    def __init__(self, num_classes):
        super(CustomModel, self).__init__()
        self.num_classes = num_classes
        # 定义网络层
        self.layer0 = layers.Dense(32, activation='relu', input_shape=(72,))   # layer0和layer1实际上不同，它们背后包含的矩阵不同
        self.layer1 = layers.Dense(32, activation='relu')
        self.layer2 = layers.Dense(num_classes, activation='softmax')
    def call(self, inputs):
        # 定义前向传播
        h1 = self.layer0(inputs)
        h2 = self.layer1(h1)
        output = self.layer2(h2)
        return output
    def summary(self):
        x = Input(shape=(72,))
        model = Model(inputs=[x], outputs=self.call(x))
        return model.summary()

In [ ]:
import numpy as np

train_x = np.random.random((1000, 72))
train_y = np.random.random((1000,10))

val_x = np.random.random((200, 72))
val_y = np.random.random((200,10))

model = CustomModel(10)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=[tf.keras.metrics.categorical_accuracy])
model.summary()
# keras.utils.plot_model(model, 'mnist_model.png')
# keras.utils.plot_model(model, 'model_info.png', show_shapes=True)

In [ ]:
model.fit(train_x, train_y, epochs=10, batch_size=100,
          validation_data=(val_x, val_y))

model.summary()
# keras.utils.plot_model(model, 'mnist_model.png')
# keras.utils.plot_model(model, 'model_info.png', show_shapes=True)

In [ ]:
test_x = np.random.random((1000, 72))
test_y = np.random.random((1000, 10))
model.evaluate(test_x, test_y, batch_size=32)
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_data = test_data.batch(32).repeat()
model.evaluate(test_data, steps=30)

**pytorch**

In [ ]:
!pip install torchmetrics

In [ ]:
import torch
import torch.nn as nn
import torchmetrics
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
print(torch.__version__)

In [ ]:
# net = nn.Sequential(nn.Linear(72, 32), nn.ReLU(), nn.Linear(32, 32), nn.ReLU(), nn.Linear(32, 10)) 

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer0 = nn.Linear(72, 32)
        self.layer1 = nn.Linear(32, 32)
        self.layer2 = nn.Linear(32, 10)
        self.actf = nn.ReLU()
    def forward(self, inputs):
        h1 = self.layer0(inputs)
        h2 = self.layer1(h1)
        h3 = self.layer2(h2)
        pred = self.actf(h3)
        return pred

# 如何读到一条数据
class CustomDataSet(Dataset):
    def __init__(self, x, y, transform=None, target_transform=None):
         self.features = x
         self.labels = y

    def __len__(self):
         return len(self.features)

    def __getitem__(self, idx):
         #  self.features = torch.autograd.Variable(self.features, requires_grad = False)
         #  self.features= self.features.data

         # 上述两行的替代方法为self.features.detach_()， 也就是将数据的梯度信息去掉
         self.features.detach_()
         return self.features[idx], self.labels[idx]


In [ ]:
net = Net()
loss = nn.CrossEntropyLoss()

optim =  torch.optim.Adam(net.parameters(), lr=0.001)

# 准备数据
train_x = torch.rand((1000, 72), requires_grad=True)
# train_x = torch.rand((1000, 72))  # 如果train_x不追踪梯度，则self.features = torch.autograd.Variable(self.features, requires_grad = False) 和 self.features= self.features.data可以去掉
train_y = torch.empty(1000, dtype=torch.long).random_(10)

val_x = torch.rand((200, 72))
val_y = torch.empty(200, dtype=torch.long).random_(10)

# 如何以batch形式读到数据
dataset = CustomDataSet(train_x, train_y)
train_dataloader = DataLoader(dataset = dataset, batch_size=100, shuffle=True, num_workers=2)

num_epochs = 10
metric = torchmetrics.Accuracy()

if __name__ == '__main__':
  for epoch in range(num_epochs):
    # for X, y in train_dataloader:
    for i, data in enumerate(train_dataloader, 0):  # 上下两个循环等价
      X, y = data  # 如果采用上一行循环，注释掉该句
      y_hat = net(X)
      l = loss(y_hat, y)
      optim.zero_grad()
      l.backward()
      # print(net[0].weight.grad)
      optim.step()
      acc = metric(y_hat.softmax(dim = -1), y)
    print(f"loss {l.detach().item()}, accracy {acc.detach().item()}. ")

In [ ]:
test_x = torch.rand((1000, 72))
test_y = torch.empty(1000, dtype=torch.long).random_(10)

num_epochs = 10
for epoch in range(num_epochs):
  test_y_hat = net(test_x)
  acc = metric(test_y_hat.softmax(dim = -1), test_y)
  print(f"accracy {acc.detach().item()}. ")